### 查看预训练模型

In [ ]:
import torchvision
dir(torchvision)

In [ ]:
from torchvision import models

In [ ]:
dir(models)

In [ ]:
# 定义一个resnet101对象
resnet101 = models.resnet101(pretrained=True)
resnet101

In [ ]:
# 定义一个Alexnet对象
alexnet = models.AlexNet()
alexnet

### 数据预处理

In [ ]:
from torchvision import transforms
transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229, 0.224, 0.225])
        ])

In [ ]:
# 查看数据集样例
from PIL import Image
img = Image.open("./data/p1ch2/bobby.jpg")

In [ ]:
dir(Image)

In [ ]:
img

In [ ]:
# 查看通过PIL打开的img格式
type(img)

In [ ]:
# 将图片进行预处理，转化成tensor，注意原图片直接打开不是tensor
img_t = transform(img)
type(img_t) # 此时图片是一个张量了

In [ ]:
# 打开模型的评估模式
resnet101.eval()

In [ ]:
# 检查img_t的维度
img_t.shape

In [ ]:
# 在img_t的第0个维度上增加1个维度，即把单图片变成批量的
import torch
batch_t = torch.unsqueeze(img_t, 0)
batch_t.shape  #此时是4维张量

In [ ]:
batch_t

In [ ]:
out = resnet101(batch_t)
out

In [ ]:
# 查看out的type
type(out)

In [ ]:
# 查看输出的维度，[1, 1000]代表1个样本，1000种class的可能性得分
out.shape

In [ ]:
# 读取data文件夹当中的class.txt，并把每一行都存到一个list当中
with open("../data/p1ch2/imagenet_classes.txt") as f:
    labels = [line.strip() for line in f.readlines()] # 这里的strip()用于去除字符串两端的空格、制表符等空白字符
print(labels)

In [ ]:
# 查看labels的维度
len(labels)

In [ ]:
# 将张量out沿着列方向(列是第1个维度，行是第0个维度)逐行寻找最大值，并获取最大值"_"和对应的class"predict"
_, predict = torch.max(out, 1)
predict  

In [ ]:
# 查看最大得分和对应class的type
print(type(_))
print(type(predict))

In [ ]:
# 查看最大得分和对应class的shape，发现都是Size([1])
print(f"_的维度是{_.shape}")
print(f"predict的维度是{predict.shape}")

In [ ]:
# 将predict转化成python数值
predict = predict.item()
predict

In [ ]:
# 查看转化后的type，发现使用item()后tensor转化为int
type(predict)

In [ ]:
# 将第1行(第1个样本)的得分标准化成百分比概率
import torch.nn.functional as F
percentage = F.softmax(out, 1)[0] * 100  # 将第1行(第1个样本)的得分标准化
percentage

In [ ]:
print(type(percentage))
print(percentage.shape)

In [ ]:
# 查看预测结果
labels[predict]

In [ ]:
# 查看预测结果的百分比概率
percentage[predict].item()

In [ ]:
# 对out进行降序排列
_, indices = torch.sort(input=out, descending=True)
indices

In [ ]:
# 查看预测的前5类
[(labels[idx], percentage[idx].item()) for idx in indices[0][:5]]